In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
#creating dataframes from CSVs
games = pd.read_csv(r"C:\Users\Brandon\Desktop\Artificial Intelligence Project\current game (outcome=1 or 0) data.csv")
#currentGameTest = pd.read_csv(r"C:\Users\Brandon\Desktop\Artificial Intelligence Project\current game test data.csv")
#futureGameTest = pd.read_csv(r"C:\Users\Brandon\Desktop\Artificial Intelligence Project\future game test data.csv")
#futureGameTrain = pd.read_csv(r"C:\Users\Brandon\Desktop\Artificial Intelligence Project\future game training data.csv")

In [3]:
X = games.drop('Player Id',axis=1).values 
y = games['Outcome'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [4]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(in_features=17, out_features=20) #applies linear transformation to incoming data (inputs)
        self.fc2 = nn.Linear(in_features=20, out_features=23) #hidden layer
        self.output = nn.Linear(in_features=23, out_features=1) #output layer
 
    #linear transformation -> ReLu -> linear transformation -> ReLu -> linear transformation
    def forward(self, x): #TODO: get single row out of x; cannot feed forward method an entire dataframe
        x = F.relu(self.fc1(x)) #limits values 0-n 
        x = F.relu(self.fc2(x))
        x = self.output(x)
        return x



In [5]:
model = ANN()
model

ANN(
  (fc1): Linear(in_features=17, out_features=20, bias=True)
  (fc2): Linear(in_features=20, out_features=23, bias=True)
  (output): Linear(in_features=23, out_features=1, bias=True)
)

In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [9]:

epochs = 100
loss_arr = []
for i in range(epochs):
    y_hat = model.forward(X_train)   
    print(torch.reshape(y_hat,[-1]).detach())
    print(y_train)
    loss = criterion(torch.reshape(y_hat,[-1]).detach(), y_train) #y_hat is full of biases and weights
    loss_arr.append(loss)
 
    if i % 10 == 0:
        print(f'Epoch: {i} Loss: {loss}')
 
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

tensor([-40.6923, -40.8835, -40.4324,  ..., -40.4158, -40.2881, -40.2659])
tensor([0, 0, 1,  ..., 1, 1, 0])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
preds = []
with torch.no_grad():
   for val in X_test:
       y_hat = model.forward(val)
       preds.append(y_hat.argmax().item())

In [ ]:
df = pd.DataFrame({'Y': y_test, 'YHat': preds})
df['Correct'] = [1 if corr == pred else 0 for corr, pred in zip(df['Y'], df['YHat'])]

In [ ]:
df['Correct'].sum() / len(df)